## 2.73 Machine Learning - Overfitting Regression Example

We are going to explore data sampled from a true (usually unknown) function that relates say house price (y in dollars) to house size (x in square feet).

Let's assume the true function is a simple curve:

$y = a.log(bx^2) + c$

And that we can collect data with a normally distributed measurement accuracy of +-d, so our measured data is sampled from this data generating model

$y = a.log(bx^2)  + c + N(0,d)$

Because of the measurement accuracy, with a fixed set of data and a perfect model in our hypothesis set we would not be able to fully resolve y due to the measurement accuracy - this is known as the irreducible error.  We will always have uncertainty.  Here is the data and the underlying data generating function:

In [6]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
from bokeh.io import gridplot
from bokeh.charts import Bar
output_notebook(hide_banner=True)

# ground truth - our function
xt = np.arange(0,20,0.1)
yt = np.log(10 * xt ** 2) + 2

n=20

# build train and test data
np.random.seed(seed=11)
x = np.random.random(n)*20
y = np.log(10 * x ** 2) + 2 + np.random.normal(scale=1, size=n)
df = pd.DataFrame(data = zip(x,y), columns=['x','y'])

p = figure()
p.circle(x=x, y=y, size=10, color='Orange')
p.line(x=xt, y=yt, color='Red', line_width=3)
show(p)

We now divide the data into a training and test set.  We then fit models of increasing flexibility to the data:

In [8]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

df_train, df_test = train_test_split(df, test_size = 0.25, random_state=1973)

def mse(a,b):
    return np.sum(np.power(np.subtract(a,b),2)) / len(a)

def fitPolynomialRegression(x_train, y_train, x_test, y_test, xt, degree=1):    
    model = make_pipeline(PolynomialFeatures(degree=degree), LinearRegression())
    model.fit(x_train, y_train) 
    return {
        'mse_train' : mse(y_train, model.predict(x_train)),
        'mse_test' : mse(y_test, model.predict(x_test)),
        'pred_yt' : model.predict(xt),
        'pred_train' : model.predict(x_train),
        'pred_test' : model.predict(x_test)
        
    } 


# fit models of increasing flexibility
mse_test = []
mse_train = []
for degree in range(1,11):
    fit = fitPolynomialRegression(
            df_train.x.reshape(-1,1), 
            df_train.y, 
            df_test.x.reshape(-1,1), 
            df_test.y, 
            xt.reshape(-1,1),
            degree)
    mse_train.append(fit['mse_train'])
    mse_test.append(fit['mse_test'])

x = range(1,11)
p = figure(plot_width=500, plot_height=300, title="Test vs Train MSE")
p.line(x=x, y=mse_train, color = 'blue', line_width = 3)
p.line(x=x, y=mse_test, color = 'red', line_width = 3)
p.y_range = Range1d(0,5)
p.xaxis.axis_label = 'Degree of Polynomial'
p.yaxis.axis_label = 'MSE'
show(p)

Let's plot the training data and overlay the models we have fit:

In [4]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
from bokeh.io import gridplot
from bokeh.charts import Bar

output_notebook()

plots_train = []
plots_test = []

for degree in range(1,11):
    fit = fitPolynomialRegression(
            df_train.x.reshape(-1,1), 
            df_train.y, 
            df_test.x.reshape(-1,1), 
            df_test.y, 
            xt.reshape(-1,1),
            degree)
    p = figure(plot_width=200, plot_height=200)
    p.title= "Train " + str(degree)
    p.title_text_font_size = '4pct'
    p.circle(x=df_train.x, y=df_train.y, size=8, color="orange", alpha=0.8)
    p.line(x=xt, y=fit['pred_yt'], color = 'red')
    p.y_range = Range1d(0,12)
    p.x_range = Range1d(0,20)
    plots_train.append(p)
    p = figure(plot_width=200, plot_height=200)
    p.title= "Test " + str(degree)
    p.title_text_font_size = '4pct'
    p.circle(x=df_test.x, y=df_test.y, size=8, color="red", alpha=0.8)
    p.line(x=xt, y=fit['pred_yt'], color = 'red')
    p.y_range = Range1d(0,12)
    p.x_range = Range1d(0,20)
    plots_test.append(p)

show(gridplot([[plots_train[0], plots_train[1], plots_train[2], plots_train[3], plots_train[4]], 
               [plots_train[5], plots_train[6], plots_train[7], plots_train[8], plots_train[9]]]))

show(gridplot([[plots_test[0], plots_test[1], plots_test[2], plots_test[3], plots_test[4]], 
               [plots_test[5], plots_test[6], plots_test[7], plots_test[8], plots_test[9]]]))


BokehJS successfully loaded.

It is clear that the higher order polynomial features help the model fit the training data very well - but when that model is applied to the test data the model performs badly.

We say the model has overfit the training data.  We do not want a model that memorises all the points in the training data - it is now learning the noise associated with the measurement error.  By definition this noise will not be the same in the test data.  